In [1]:
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
    n_head=10,
    n_layer=12,
    n_embd=760,
    )

model = GPT2LMHeadModel(config)

/home/sam/pretraining/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import pretrain as pt

pages = [
    random.randint(1, pt.dataset.SubsetFalconLoader.max_pages)
    for _ in range(1)
]
batches = list(
    pt.dataset.SubsetFalconLoader(
        batch_size=1,
        sequence_length=10,
        pages=pages,
    )
)


In [3]:
import math

model.to('cpu')
model.eval()

# Iterate over each page and corresponding batches
losses = []
for batch in batches:
    try:
        inputs = batch.to('cpu')
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss.item()  # Extract scalar loss value
        losses.append(loss)
    except Exception as e:
        print(f"Exception occurred: {e}")
        losses.append(math.inf)  # Use infinity to indicate failure

print(losses)

In [ ]:
import math
import torch

model.to('cpu')
model.eval()

# Iterate over each page and corresponding batches
losses = []
for batch in batches:
    try:
        inputs = batch.to('cpu')
        logits = model(inputs).logits
        print(f"Batch logits: {logits}")

        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = inputs[..., 1:].contiguous()
        # Flatten the tokens
        loss_fct = torch.nn.CrossEntropyLoss()
        shift_logits = shift_logits.view(-1, model.config.vocab_size)
        shift_labels = shift_labels.view(-1)
        loss = loss_fct(shift_logits, shift_labels).item()
    
        losses.append(loss)
        print(f"Batch loss: {loss}")
    except Exception as e:
        print(f"Exception occurred: {e}")
        losses.append(math.inf)  # Use infinity to indicate failure

print(losses)